In [135]:
""" Employed bee phase function

Input
- population of solutions (food sources)
- lower and upper bounds of variables
- current trial of solutions

Output
- new evolved population
- new fitness values
- updated trials of solutions in population: 
    When original solution has failed to generate better solution, trial counter is increased by 1 unit
    When better solution has been found, the trial counter for this new solution is set to zero

"""

function employed_bee_phase(population, bounds_lower, bounds_upper, trial)
    population_new = []
    
    # create new food sources
    for i in 1:size(population)[1]
        solution = population[i, :][1]
        solution_new = create_newsolution(solution, population, bounds_lower, bounds_upper)     
        append!(population_new, [solution_new])
    end
    
    # evaluate fitness old and new population
    objective_values_old = compute_objective(population, false)
    fitness_old = compute_fitness(objective_values_old)
    objective_values_new = compute_objective(population_new, false)
    fitness_new = compute_fitness(objective_values_new)

    # perform greedy selection
    population_new_evolved = []
    fitness_new_evolved = []
    objective_new_evolved = []
    
    for j in 1:size(population)[1]
        if fitness_new[j] > fitness_old[j]
            append!(population_new_evolved, [population_new[j]])
            append!(fitness_new_evolved, fitness_new[j])
            append!(objective_new_evolved, objective_values_new[j])
            trial[j]=0
        else 
           append!(population_new_evolved, [population[j]]) 
           append!(fitness_new_evolved, fitness_old[j])
            append!(objective_new_evolved, objective_values_old[j])
            trial[j] += 1
        end
    end
    
    return population_new_evolved, fitness_new_evolved, objective_new_evolved, trial
end	


employed_bee_phase (generic function with 2 methods)

In [136]:
# input employed bee phase
trial = zeros(size(population)[1])
bounds_lower = [-5,-5,-5,-5]
bounds_upper = [5,5,5,5]
D=4
n=9
population = initialize_population(D, bounds_lower, bounds_upper, n)

# execute employed bee phase
pop_new_evolved, fitness_new_evolved, objective_new_evolved, trial_new = employed_bee_phase(population, bounds_lower, bounds_upper, trial)


# visualize output in dataframe
# import Pkg; Pkg.add("DataFrames")
using DataFrames
df = DataFrame(old_population = population, new_population=pop_new_evolved, fitness=fitness_new_evolved, objective=objective_new_evolved, trial=trial_new)

,old_population,new_population,fitness,objective,trial
,Any,Any,Any,Any,Float64
1,"[0, -5, 4, 4]","[0, -5, 4, 4]",0.0172414,57,1.0
2,"[-2, 0, 4, 3]","[-2.0, 0.0, 3.77212, 3.0]",0.0354247,27.2289,0.0
3,"[5, -1, 5, 5]","[5, -1, 5, 5]",0.012987,76,1.0
4,"[-1, 0, 3, 3]","[-1, 0, 3, 3]",0.05,19,1.0
5,"[0, 5, -3, 1]","[0, 5, -3, 1]",0.0277778,35,1.0
6,"[-2, 0, -4, -5]","[-2, 0, -4, -5]",0.0217391,45,1.0
7,"[-4, 4, 0, -3]","[-4, 4, 0, -3]",0.0238095,41,1.0
8,"[-4, -4, -2, -5]","[-4.0, -4.0, -2.0, -3.75061]",0.0195821,50.067,0.0
9,"[1, 3, 2, 0]","[1, 3, 2, 0]",0.0666667,14,1.0


### Andere functies nodig voor employed bee phase

In [88]:
function compute_objective(input, vector=true)
    if vector == true
        objective = sum(input[i]^2 for i in 1:length(input))
        output = objective
    else
        objectives_population = []
        
        for j in 1:size(input)[1]
            food_source = input[j, :]
            #objective = sum(food_source[i]^2 for i in 1:length(food_source))
            objective = sum(sum(food_source[i].^2 for i in 1:length(food_source)))
            append!(objectives_population, objective)
        end
        
        output = objectives_population
    end
    
    return output
end

compute_objective (generic function with 2 methods)

In [91]:
function compute_fitness(objective_values)
    fitness_values = []
    
    for i in 1:length(objective_values)
        objective_value = objective_values[i]
        
        if objective_value >= 0
            fitness = 1/(1+objective_value)
     
        else
            fitness = 1+abs(objective_value)
        end
        
        append!(fitness_values, fitness)
    end
    return fitness_values
end	

compute_fitness (generic function with 1 method)

In [114]:
function initialize_population(D, bounds_lower, bounds_upper, n)
    population = []   
    for i in 1:n
        food_source = collect(rand(bounds_lower[i]:bounds_upper[i]) for i in 1:D)
        append!(population, [food_source])
    end
        
    return population
end	
bounds_lower = [-5,-5,-5,-5]
bounds_upper = [5,5,5,5]
D=4
n=9
population = initialize_population(D, bounds_lower, bounds_upper, n)

9-element Array{Any,1}:
 [-3, -2, -5, 2]
 [4, 0, 2, 3]
 [5, -1, -1, 3]
 [1, 3, -2, 0]
 [5, -2, -2, -3]
 [3, -4, -3, -5]
 [2, 5, 1, -4]
 [-3, 0, 4, 2]
 [3, 0, -1, -1]

In [83]:
function create_newsolution(solution, population, bounds_lower, bounds_upper)
    
    # select random variable to change       
    randomvar1_index = rand(1:length(solution), 1)
        
    # select partner solution to generate new solution        
    randompartner_index = rand(1:size(population)[1], 1)
    
    # select random variable in partner solution to exchange with
        
    randompartner = population[randompartner_index, :][1]
    randomvar2_index = rand(1:length(randompartner), 1)
        
    # create new food location
    phi = rand()*2-1 #random number between -1 and 1     
    global solution_new = float(deepcopy(solution))
    a = solution[randomvar1_index] 
    b = randompartner[randomvar2_index]
    solution_new[randomvar1_index] = a + phi*(a - b)
    
    # check if lower bound is violated
    if solution_new[randomvar1_index] < bounds_lower[randomvar1_index] 
        solution_new[randomvar1_index] = bounds_lower[randomvar1_index]
    end
    
    # check if upper bound is violated
    if solution_new[randomvar1_index] > bounds_upper[randomvar1_index]
        solution_new[randomvar1_index] = bounds_upper[randomvar1_index]
    end
        
    return solution_new
end	

create_newsolution (generic function with 1 method)